In [1]:
import os
import numpy as np
import pandas as pd
from importlib import reload

from PATH import data_path

In [2]:
from sklearn import decomposition

In [3]:
import utils
reload(utils)

<module 'utils' from '/home/ZwZ/script/HER2_prediction/utils.py'>

In [4]:
pj = os.path.join

## MISSING VALUE

In [7]:
DF=pd.read_csv(pj(data_path,'TCGA_BRCA_Her2.csv'))

In [10]:
DF.iloc[:,1].unique()

array(['Negative', 'Not Available', 'Positive', nan], dtype=object)

## read expression matrix

In [64]:
mat1,header1,gene1 = utils.read_expression_matrix(pj(data_path,'symbol-1.txt'))
mat2,header2,gene2 = utils.read_expression_matrix(pj(data_path,'symbol-2.txt'))

gene_DF1=pd.DataFrame({'symbol':np.array(gene1),
                       'idx':range(len(gene1))})
gene_DF2=pd.DataFrame({'symbol':np.array(gene2),
                       'idx':range(len(gene2))})

In [126]:
merged_DF=gene_DF1.merge(gene_DF2,left_on=['symbol'],right_on=['symbol'],indicator=True)

In [132]:
gene_DF1.symbol.unique()

array(['TCEA1', 'SORT1', 'ADAM19', ..., 'BPIFB4', 'CCDC78', 'ZNF579'],
      dtype=object)

### drop duplicate

In [152]:
keep1,discard1 = utils.keep_identical(gene1)
keep2,discard2 = utils.keep_identical(gene2)

In [158]:
gene1=gene1[keep1]
gene2=gene2[keep2]

In [161]:
gene_DF1=pd.DataFrame({'symbol':np.array(gene1),
                       'idx':range(len(gene1))})
gene_DF2=pd.DataFrame({'symbol':np.array(gene2),
                       'idx':range(len(gene2))})
merged_DF=gene_DF1.merge(gene_DF2,how='outer',left_on=['symbol'],right_on=['symbol'],indicator=True)

In [164]:
resort_gene2=merged_DF.idx_y.values

In [166]:
np.sum(gene2[resort_gene2] == gene1)

19645

## re-structure gene

In [168]:
mat1 = mat1[keep1,:]

In [169]:
mat2 = mat2[keep2,:]

In [175]:
mat2=mat2[resort_gene2,:]

In [176]:
whole_mat = np.concatenate([mat1.T,mat2.T],axis=0)

In [177]:
whole_mat.shape

(1222, 19645)

### CHECK

In [184]:
mat1,header1,gene1 = utils.read_expression_matrix(pj(data_path,'symbol-1.txt'))
mat2,header2,gene2 = utils.read_expression_matrix(pj(data_path,'symbol-2.txt'))

In [187]:
np.sum(gene1[keep1] == gene2[keep2][resort_gene2])

19645

In [188]:
mat1 = mat1[keep1]
mat2 = mat2[keep2][resort_gene2]

In [213]:
DF1 = pd.DataFrame(mat1,columns=header1)
DF1 = DF1.set_index('id')
DF2 = pd.DataFrame(mat2,columns=header2)
DF2 = DF2.set_index('id')

In [217]:
whole=DF1.T.append(DF2.T)
whole=whole.astype(np.float32)

In [231]:
np.sum(whole_mat == whole.values)

24006190

In [232]:
whole_mat.size

24006190

## | SAVE to new FILE  |

In [178]:
np.save(os.path.join(data_path,'WHOLE_MATRIX.npy'),whole_mat)

In [228]:
whole.to_csv(pj(data_path,'whole_matrix.csv'))

## Data

In [5]:
from utils import read_data_DF,read_label

In [6]:
DF = read_data_DF(data_path)
label = read_label(data_path)

In [13]:
label.status.unique()

array(['Negative', 'Not Available', 'Positive', nan], dtype=object)

In [35]:
merged_df = DF.merge(label,left_index=True,right_on=['sample'])
merged_df.index = range(merged_df.shape[0])
to_pred_DF=merged_df[merged_df.status == 'Not Available'].append(merged_df[merged_df.status.isnull()])

In [49]:
to_pred_DF.iloc[:,:-1].set_index('sample').to_csv(pj(data_path,'to_pred_DF.csv'),index=True)

## the collection of ER and PR label